# Imports

In [2]:
!pip install python-terrier
!pip install ir_datasets
#!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

     |████████████████████████████████| 71kB 4.6MB/s 
     |████████████████████████████████| 1.1MB 13.9MB/s 
     |████████████████████████████████| 71kB 6.3MB/s 
     |████████████████████████████████| 2.8MB 45.7MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 133kB 47.5MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 81kB 6.6MB/s 
     |████████████████████████████████| 81kB 6.6MB/s 
     |████████████████████████████████| 1.8MB 43.9MB/s 
     |████████████████████████████████| 5.5MB 48.4MB/s 
     |████████████████████████████████| 81kB 7.0MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.4.0-cp37-none-any.whl size=76208 sha256=cdba06f2f5435f582c2b3cc70feb75cdd4911dc15e1d45c5dc61ff93d8380534
  Stored in directory: /root/.cache/pip/wheels/1a/08/00/1eb79fd13e6095132bf00bd92f803e90797813f40429485461
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9

     |████████████████████████████████| 645kB 11.2MB/s 
ERROR: myst-parser 0.13.5 has requirement sphinx<4,>=2, but you'll have sphinx 1.8.5 which is incompatible.
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb

# Data and Indexing

In [4]:
import pyterrier as pt
pt.init()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


terrier-assemblies 5.4  jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.5  jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.4.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [5]:
dataset = pt.get_dataset("trec-deep-learning-passages")
def msmarco_generate():
    with pt.io.autoopen(dataset.get_corpus()[0], 'rt') as corpusfile:
        for l in corpusfile:
            docno, passage = l.split("\t")
            yield {'docno' : docno, 'text' : passage}

In [6]:
iter_indexer = pt.IterDictIndexer("./passage_index")
indexref = iter_indexer.index(msmarco_generate(), meta={'docno' : 20, 'text': 4096})

collection.tar.gz: 100%|██████████| 987M/987M [10:51<00:00, 1.59MiB/s]


09:50:27.499 [ForkJoinPool-1-worker-3] WARN  o.t.structures.indexing.Indexer - Adding an empty document to the index (500080) - further warnings are suppressed
10:22:44.756 [ForkJoinPool-1-worker-3] WARN  o.t.structures.indexing.Indexer - Indexed 5 empty documents


In [7]:
index = pt.IndexFactory.of(indexref)

# Experiments with Prob. Models and LTR

**Prob Models**

In [8]:
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
DPH_br = pt.BatchRetrieve(index, wmodel="DPH") 

In [9]:
pt.Experiment(
    [tf_idf, bm25, DPH_br],
    dataset.get_topics('test-2019'),
    dataset.get_qrels('test-2019'),
    eval_metrics=["map", "recip_rank", "ndcg"],
    names = ['TF_IDF', 'BM25', 'DPH']
)

msmarco-test2019-queries.tsv.gz: 100%|██████████| 4.18k/4.18k [00:00<00:00, 1.61MiB/s]


2019qrels-docs.txt: 100%|██████████| 183k/183k [00:00<00:00, 712kiB/s] 


10:31:49.959 [main] WARN  o.t.structures.CompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 2.2 GiB of memory would be required.


,name,map,recip_rank,ndcg
0,TF_IDF,0.369486,0.795028,0.593198
1,BM25,0.370004,0.795028,0.593433
2,DPH,0.372805,0.799962,0.590368


**LTR**

In [10]:
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25",
  features=["SAMPLE", "WMODEL:TF_IDF", "WMODEL:PL2", "WMODEL:IFB2", "WMODEL:Hiemstra_LM", "WMODEL:LGD", "WMODEL:DPH", "WMODEL:In_expC2", "WMODEL:ML2"])

In [11]:
train_topics = dataset.get_topics('train')
train_qrels = dataset.get_qrels('train')
test_topics = dataset.get_topics('test-2019')
test_qrels = dataset.get_qrels('test-2019')

queries.tar.gz: 100%|██████████| 18.0M/18.0M [00:04<00:00, 3.98MiB/s]


qrels.train.tsv: 100%|██████████| 10.1M/10.1M [00:06<00:00, 1.64MiB/s]


In [ ]:
rf = RandomForestRegressor(n_estimators=400)
ltr = pipeline >> pt.ltr.apply_learned_model(rf)
ltr.fit(dataset.get_topics('train'), dataset.get_qrels('train'))

queries.tar.gz: 100%|██████████| 18.0M/18.0M [00:01<00:00, 15.4MiB/s]
qrels.train.tsv:   0%|          | 0.00/10.1M [00:00<?, ?iB/s]

qrels.train.tsv: 100%|██████████| 10.1M/10.1M [00:01<00:00, 9.72MiB/s]


In [ ]:
pt.Experiment([bm25, tf_idf, ltr, DPH_br], dataset.get_topics('test-2019'),
    dataset.get_qrels('test-2019'), eval_metrics=["trec_eval"], names=["BM25 Baseline", "TF_IDF", "LTR", "DPH"])

NameError: ignored

In [ ]:
lmart = lgb.LGBMRanker(task="train",
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=100,
    max_bin=255,
    num_leaves=7,
    objective="lambdarank",
    metric="ndcg",
    #ndcg_eval_at=[1, 3, 5, 10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=10)
lmart_ltr = pipeline >> pt.ltr.apply_learned_model(lmart, form="ltr")
lmart_ltr.fit(train_topics, train_qrels)

In [ ]:
pt.Experiment([bm25, tf_idf, lmart_ltr, DPH_br], dataset.get_topics('test-2019'),
    dataset.get_qrels('test-2019'), eval_metrics=["trec_eval"], names=["BM25 Baseline", "TF_IDF", "Lmart_LTR", "DPH"])